In [11]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [10]:
#pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


# This is the first section, transform data from Wikipedia to data frame: df

In [122]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [123]:
### get requests from the url
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

results = soup.find(id='mw-content-text')
### find the table contents from the webpage
tab = soup.find("table",{"class":"wikitable sortable"})
print(tab)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [133]:
### Get data from each row on weiki; 
### append each row column data from weiki to data frame df_unclean;
### Try to remove first row of data, or remove the title when extracting data. .....fixed
### Try to remove \n.....fixed
### Try to remove not assign row for Borough

table_rows = tab.find_all('tr')
empty_df = []
for tr in table_rows:
    td = tr.find_all('td')
    row = ([tr.text for tr in td])
    empty_list = []       #### Create empty list to store value which shall removed \n by using .strip()
    for elems in row:
        empty_list.append(elems.strip())
    empty_df.append(empty_list)
    
df_unclean = pd.DataFrame(empty_df, columns=["PostalCode", "Borough", "Neighborhood"])
#df_unclean.drop(df_unclean.index[0])
df_unclean['Borough'].replace({'Not assigned': np.nan}, inplace = True)
df_unclean
df_unclean.dropna(axis = 0, how='any', inplace = True)
df = df_unclean.reset_index(drop = True)  ### remove the first row of the title, reset index
#df["Neighborhood"]= df["Neighborhood"].str.split("and", expand= True)

print(df.shape)
df

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# This is the second section, that add latitude and longtitude to the dataframe

In [135]:
df1 = pd.read_csv('http://cocl.us/Geospatial_data')  ### read the csv file by using pandas

In [143]:
df1.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)  ### rename the column name
df_Geo = df.join(df1.set_index('PostalCode'), on = 'PostalCode')
df_Geo.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
